# Setup

In [7]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/My\ Drive/CLIPProj

Mounted at /content/drive
/content/drive/My Drive/CLIPProj


In [8]:
# Install dependencies
# !pip freeze | grep python-dotenv  # check existing libraries
!pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 94.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.6/207.6 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 339.9/339.9 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 4.5 MB/s eta 0:00:00


In [9]:
# Library Imports
import os
from dotenv import load_dotenv

import numpy as np
import matplotlib.pyplot as plt

import torch
import torchvision
import lightning as L
from lightning.pytorch.callbacks import ModelCheckpoint, EarlyStopping

from transformers import CLIPProcessor, CLIPVisionModel

In [ ]:
# General Setup


# Data

In [10]:
# Caltech 101 Data
data = torchvision.datasets.Caltech101('data', transform=torchvision.transforms.ToTensor, download=True)

Downloading...
From (original): https://drive.google.com/uc?id=137RyRjvTBkBiIfeYBNZBtViDHQ6_Ewsp
From (redirected): https://drive.usercontent.google.com/download?id=137RyRjvTBkBiIfeYBNZBtViDHQ6_Ewsp&confirm=t&uuid=913229c6-b230-4490-8bc9-75927ae5184d
To: /content/drive/MyDrive/CLIPProj/data/caltech101/101_ObjectCategories.tar.gz
100%|██████████| 132M/132M [00:01<00:00, 78.6MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=175kQy3UsZ0wUEHZjqkUDdNVssr7bgh_m
From (redirected): https://drive.usercontent.google.com/download?id=175kQy3UsZ0wUEHZjqkUDdNVssr7bgh_m&confirm=t&uuid=7f5c48a8-9f45-4c48-8658-eea4788353ed
To: /content/drive/MyDrive/CLIPProj/data/caltech101/Annotations.tar
100%|██████████| 14.0M/14.0M [00:00<00:00, 95.9MB/s]


In [ ]:
from lightning.pytorch.loggers import WandbLogger
import wandb